1. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
3. Вывести сравнение полученных моделей по метрикам бизнеса по показателям с урока
    - стоимость лечения 15000р, если сделали тест и начали лечить вовремя
    - стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы
    - стоимость теста 1400р
4. Сделать выводы о том, какая модель справилась с задачей лучше других

In [53]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [1]:
!wget 'https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G' -O train_case2.csv

--2022-03-17 11:14:04--  https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G
Resolving drive.google.com (drive.google.com)... 172.217.204.139, 172.217.204.100, 172.217.204.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.204.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r42qktg86m548llohevek05cadttu0up/1647515625000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download [following]
--2022-03-17 11:14:04--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r42qktg86m548llohevek05cadttu0up/1647515625000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download
Resolving doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-00-c0-docs.googleusercontent.com (doc-00-c0

In [20]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
   
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

In [6]:
df = pd.read_csv('train_case2.csv', ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], random_state=0)

In [68]:
def model_metric_table(model, X_train, X_test, y_train, y_test):

  model.fit(X_train, y_train)
  y_score = model.predict_proba(X_test)[:, 1]

  b = 1
  precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
  fscore = (1 +b**2) * (precision * recall) / (b**2 * precision + recall)

  ix = np.argmax(fscore)

  return {'precision': precision[ix],
          'recall': recall[ix],
          'f1_score': fscore[ix],
          'roc_auc': roc_auc_score(y_test, y_score),
          'best_threshold': thresholds[ix]}

In [69]:
metric_table = pd.DataFrame(columns=['precision', 'recall', 'f1_score', 'roc_auc', 'best_threshold'])

Логистическая регрессия

In [70]:
classifier_logr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])

logr = model_metric_table(classifier_logr, X_train, X_test, y_train, y_test)
metric_table = metric_table.append(logr, ignore_index=True)
cv_scores = cross_val_score(classifier_logr, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score LogReg is {cv_score:.4f}+-{cv_score_std:.4f}')

CV score LogReg is 0.7865+-0.0044


Градиентный бустинг

In [71]:
classifier_gb = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])

gb = model_metric_table(classifier_gb, X_train, X_test, y_train, y_test)
metric_table = metric_table.append(gb, ignore_index=True)
cv_scores = cross_val_score(classifier_gb, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score GradBoost is {cv_score:.4f}+-{cv_score_std:.4f}')

CV score GradBoost is 0.8023+-0.0031


Случайный лес

In [72]:
classifier_randforest = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

randforest = model_metric_table(classifier_randforest, X_train, X_test, y_train, y_test)
metric_table = metric_table.append(randforest, ignore_index=True)
cv_scores = cross_val_score(classifier_randforest, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score RandForest is {cv_score:.4f}+-{cv_score_std:.4f}')

CV score RandForest is 0.7744+-0.0036


Таблица

In [73]:
metric_table

,precision,recall,f1_score,roc_auc,best_threshold
0,0.647431,0.837558,0.730323,0.784035,0.386937
1,0.697848,0.788134,0.740248,0.802615,0.394947
2,0.642669,0.815553,0.718863,0.771037,0.350000


Сравнение полученных моделей по метрикам бизнеса

In [84]:
def buisnes_cost(model, X_train, X_test, y_train, thresholds):

  model.fit(X_train, y_train)
  y_score = model.predict_proba(X_test)[:, 1]

  rubl_test = 1400
  rubl_early_treatment = 15000
  rubl_late_treatment = 20000
  
  cnf_matrix = confusion_matrix(y_test, y_score > thresholds)
  TN = cnf_matrix[0][0]
  FN = cnf_matrix[1][0]
  TP = cnf_matrix[1][1]
  FP = cnf_matrix[0][1]

  rubl_1 = (FN + TP) * rubl_late_treatment
  rubl_test_all = np.sum(cnf_matrix) * rubl_test + (FN + TP) * rubl_early_treatment
  rubl_ML = (FP + TP) * rubl_test + FN * rubl_late_treatment + TP * rubl_early_treatment

  print('расходы, если не делать тест и ждать симптомов:', rubl_1)
  print('расходы, если делать тест всем и лечить группу большых:', rubl_test_all)
  print('расходы, если использовать МЛ:', rubl_ML)
  print('Расходы "решение не делать тесты - расходы с МL:"', rubl_1 - rubl_ML)
  print('Расходы "решение не делать тесты - расходы сделать тесты всем":', rubl_1 - rubl_test_all)
  print('Расходы "сделать тесты всем - решение делать тесты ML":', rubl_test_all - rubl_ML)

In [98]:
logres_cost = buisnes_cost(classifier_logr, X_train, X_test, y_train, metric_table.iloc[0, 4])
logres_cost

расходы, если не делать тест и ждать симптомов: 173600000
расходы, если делать тест всем и лечить группу большых: 154700000
расходы, если использовать МЛ: 152974200
Расходы "решение не делать тесты - расходы с МL:" 20625800
Расходы "решение не делать тесты - расходы сделать тесты всем": 18900000
Расходы "сделать тесты всем - решение делать тесты ML": 1725800


In [99]:
gb_cost = buisnes_cost(classifier_gb, X_train, X_test, y_train, metric_table.iloc[1, 4])
gb_cost

расходы, если не делать тест и ждать симптомов: 173600000
расходы, если делать тест всем и лечить группу большых: 154700000
расходы, если использовать МЛ: 153122800
Расходы "решение не делать тесты - расходы с МL:" 20477200
Расходы "решение не делать тесты - расходы сделать тесты всем": 18900000
Расходы "сделать тесты всем - решение делать тесты ML": 1577200


In [100]:
randforest_cost = buisnes_cost(classifier_randforest, X_train, X_test, y_train, metric_table.iloc[2, 4])
randforest_cost

расходы, если не делать тест и ждать симптомов: 173600000
расходы, если делать тест всем и лечить группу большых: 154700000
расходы, если использовать МЛ: 153699400
Расходы "решение не делать тесты - расходы с МL:" 19900600
Расходы "решение не делать тесты - расходы сделать тесты всем": 18900000
Расходы "сделать тесты всем - решение делать тесты ML": 1000600


Выводы 

Градиентный бустинг оказался наиболее точный во всех показателях, кроме recall 
